In [ ]:
import urllib
import urllib.request as request
from io import BytesIO
import pandas as pd
import json
import tsfresh
from scipy import stats

import time

import matplotlib.pyplot as plt

In [ ]:
# tsfresh - for timeseries
# Grainger causality
# Dynmaic Time Warping - https://en.wikipedia.org/wiki/Dynamic_time_warping for series similarity
# http://didawiki.cli.di.unipi.it/lib/exe/fetch.php/dm/time_series_from_keogh_tutorial.pdf
# https://datascience.stackexchange.com/questions/15812/check-similarity-between-time-series
# https://datascience.stackexchange.com/questions/29283/how-to-find-similar-time-series

In [ ]:
lunar_start = 'https://api.lunarcrush.com/v2?'

# queryStringParams = {
#   'data':'assets',
#   'key':'hnvavbmd2fj62h5kzd2imb',
#   'symbol':[','.join(list(pd.DataFrame(data['data'])['symbol'])[:2])],
#   #'symbol':"btc,ltc",
#   'data_points': 3,
#   'interval':'day'
# }

# queryStringParams = {
#   'data':'global',
#   'key':'hnvavbmd2fj62h5kzd2imb',
#   'data_points': 3
# }

# ','.join(list(pd.DataFrame(data['data'])['symbol']))
# to get list of assets
queryStringParams = {
  'data':'meta',
  'key':'hnvavbmd2fj62h5kzd2imb',
  'data_points': 1
}


encodedParams = urllib.parse.urlencode(queryStringParams)

#url = "data=global&key=hnvavbmd2fj62h5kzd2imb&data_points=24"
#url = "data=assets&key=hnvavbmd2fj62h5kzd2imb&symbol=DOGE&data_points=365&interval=day"

# https://stackoverflow.com/questions/31778800/how-can-i-make-a-post-request-on-python-with-urllib3
# https://stackoverflow.com/questions/7933417/how-do-i-set-headers-using-pythons-urllib
# https://www.pluralsight.com/guides/importing-data-from-json-resource-with-python

with request.urlopen(lunar_start + encodedParams) as response:
        if response.getcode() == 200:
            source = response.read()        
            data_coins = json.loads(source)
            print(data_coins['usage'])
        else:
            print('An error occurred while attempting to retrieve data from the API.')

{'day': 1, 'month': 465}


In [ ]:
list_assets = [','.join(list(pd.DataFrame(data_coins['data'])['symbol']))]
#list_assets[0].lower().split(',')[:10]
list_assets

['BTC,ETH,XRP,LTC,BCH,BNB,USDT,EOS,BSV,XLM,ADA,TRX,XMR,LEO,DASH,NEO,MIOTA,LINK,ATOM,XTZ,ETC,XEM,MKR,ONT,CRO,ZEC,VSYS,USDC,DOGE,BTG,BAT,VET,EGT,QTUM,DCR,HT,HEDG,OMG,TUSD,PAX,RVN,NANO,LSK,BTT,BCD,HOT,KCS,WAVES,ZRX,NPXS,REP,ICX,DGB,MONA,KMD,HC,BTS,AOA,IOST,BCN,BTM,LAMB,SC,THETA,EKT,INB,NEX,XIN,NRG,QNT,ZIL,XVG,GXC,AE,WTC,REN,STEEM,DAI,ABBC,NET,XZC,SNT,MCO,ARDR,ENJ,GNT,VEST,MAID,ELF,NEXO,SOLVE,UUU,MXM,CCCX,JCT,ETP,XMX,RIF,STRAX,PAI,ZEN,FSN,GRIN,FX,ETN,PPT,NAS,SAN,DENT,ELA,ECOREAL,MANA,QASH,LRC,FTM,FST,ARK,ODE,AION,FCT,ORBS,RDD,NULS,DGD,VERI,LA,TOMO,EURS,KNC,ENG,IGNIS,WICC,BCZERO,LOOM,TRUE,CMT,SNX,MOAC,POWR,BNT,QKC,WAN,TFUEL,BIX,CELR,BEAM,PIVX,NEW,ONE,AGVC,HYN,ABT,APL,CRPT,C20,MATIC,ANT,IOTX,STORJ,DGTX,BHP,POLY,TEL,BZ,FET,BRD,PLC,BOX,RLC,GRS,TT,LXT,GAS,SYS,NXT,META,DRGN,TTC,ITC,CTXC,BFT,FUN,BOTX,SKY,CVC,GBYTE,UGAS,GNO,EDO,MITH,DAC,ROX,KAN,VTC,MEDX,MTL,CSC,SEELE,TRXC,CVNT,PAY,HPB,ETZ,TNT,NXS,ANKR,BCV,RUFF,AGI,DAPS,UTK,EMC2,DTR,LTO,PART,ADN,NKN,FOAM,VITAE,MFT,TKN,QRL,MTC,HYC,CND,CLAM,WIX,GTO,I

In [ ]:
numberOfCoinsToGet = 50 # per batch
v = [','.join(list(pd.DataFrame(data_coins['data']).head(numberOfCoinsToGet)['symbol']))]
assets_low = [x.lower() for x in v]
assets_low[0]

'btc,eth,xrp,ltc,bch,bnb,usdt,eos,bsv,xlm,ada,trx,xmr,leo,dash,neo,miota,link,atom,xtz,etc,xem,mkr,ont,cro,zec,vsys,usdc,doge,btg,bat,vet,egt,qtum,dcr,ht,hedg,omg,tusd,pax,rvn,nano,lsk,btt,bcd,hot,kcs,waves,zrx,npxs'

In [ ]:
# https://stackoverflow.com/questions/47239332/take-the-sum-of-every-n-rows-in-a-pandas-series
coin_groups = pd.DataFrame(data_coins['data'])['symbol'].values[:-11].reshape(-1, numberOfCoinsToGet)
coin_groups[:5]

array([['BTC', 'ETH', 'XRP', 'LTC', 'BCH', 'BNB', 'USDT', 'EOS', 'BSV',
        'XLM', 'ADA', 'TRX', 'XMR', 'LEO', 'DASH', 'NEO', 'MIOTA',
        'LINK', 'ATOM', 'XTZ', 'ETC', 'XEM', 'MKR', 'ONT', 'CRO', 'ZEC',
        'VSYS', 'USDC', 'DOGE', 'BTG', 'BAT', 'VET', 'EGT', 'QTUM',
        'DCR', 'HT', 'HEDG', 'OMG', 'TUSD', 'PAX', 'RVN', 'NANO', 'LSK',
        'BTT', 'BCD', 'HOT', 'KCS', 'WAVES', 'ZRX', 'NPXS'],
       ['REP', 'ICX', 'DGB', 'MONA', 'KMD', 'HC', 'BTS', 'AOA', 'IOST',
        'BCN', 'BTM', 'LAMB', 'SC', 'THETA', 'EKT', 'INB', 'NEX', 'XIN',
        'NRG', 'QNT', 'ZIL', 'XVG', 'GXC', 'AE', 'WTC', 'REN', 'STEEM',
        'DAI', 'ABBC', 'NET', 'XZC', 'SNT', 'MCO', 'ARDR', 'ENJ', 'GNT',
        'VEST', 'MAID', 'ELF', 'NEXO', 'SOLVE', 'UUU', 'MXM', 'CCCX',
        'JCT', 'ETP', 'XMX', 'RIF', 'STRAX', 'PAI'],
       ['ZEN', 'FSN', 'GRIN', 'FX', 'ETN', 'PPT', 'NAS', 'SAN', 'DENT',
        'ELA', 'ECOREAL', 'MANA', 'QASH', 'LRC', 'FTM', 'FST', 'ARK',
        'ODE', 'AION', 'FCT', '

In [ ]:
for group in coin_groups:
    print(','.join(group).lower())
    print()

btc,eth,xrp,ltc,bch,bnb,usdt,eos,bsv,xlm,ada,trx,xmr,leo,dash,neo,miota,link,atom,xtz,etc,xem,mkr,ont,cro,zec,vsys,usdc,doge,btg,bat,vet,egt,qtum,dcr,ht,hedg,omg,tusd,pax,rvn,nano,lsk,btt,bcd,hot,kcs,waves,zrx,npxs

rep,icx,dgb,mona,kmd,hc,bts,aoa,iost,bcn,btm,lamb,sc,theta,ekt,inb,nex,xin,nrg,qnt,zil,xvg,gxc,ae,wtc,ren,steem,dai,abbc,net,xzc,snt,mco,ardr,enj,gnt,vest,maid,elf,nexo,solve,uuu,mxm,cccx,jct,etp,xmx,rif,strax,pai

zen,fsn,grin,fx,etn,ppt,nas,san,dent,ela,ecoreal,mana,qash,lrc,ftm,fst,ark,ode,aion,fct,orbs,rdd,nuls,dgd,veri,la,tomo,eurs,knc,eng,ignis,wicc,bczero,loom,true,cmt,snx,moac,powr,bnt,qkc,wan,tfuel,bix,celr,beam,pivx,new,one,agvc

hyn,abt,apl,crpt,c20,matic,ant,iotx,storj,dgtx,bhp,poly,tel,bz,fet,brd,plc,box,rlc,grs,tt,lxt,gas,sys,nxt,meta,drgn,ttc,itc,ctxc,bft,fun,botx,sky,cvc,gbyte,ugas,gno,edo,mith,dac,rox,kan,vtc,medx,mtl,csc,seele,trxc,cvnt

pay,hpb,etz,tnt,nxs,ankr,bcv,ruff,agi,daps,utk,emc2,dtr,lto,part,adn,nkn,foam,vitae,mft,tkn,qrl,mtc,hyc,cnd,clam,wix,gto

In [ ]:
len(coin_groups)

58

In [ ]:
df_full_z = pd.DataFrame()
df_full_price = pd.DataFrame()

for idx, group in enumerate(coin_groups):

    
    print(idx, " of ", len(coin_groups))
    
    assets_low = ','.join(group).lower()
    print(assets_low)

    queryStringParams = {
      'data':'assets',
      'key':'hnvavbmd2fj62h5kzd2imb',
      'symbol':assets_low,
      'data_points': 720,
      'interval':'day',
      'time_series_indicators':'close',
      #'start': 1627852171,
      #'end': 1630444171
    }


    encodedParams = urllib.parse.urlencode(queryStringParams)

    #url = "data=global&key=hnvavbmd2fj62h5kzd2imb&data_points=24"
    #url = "data=assets&key=hnvavbmd2fj62h5kzd2imb&symbol=DOGE&data_points=365&interval=day"

    # https://stackoverflow.com/questions/31778800/how-can-i-make-a-post-request-on-python-with-urllib3
    # https://stackoverflow.com/questions/7933417/how-do-i-set-headers-using-pythons-urllib
    # https://www.pluralsight.com/guides/importing-data-from-json-resource-with-python

    with request.urlopen(lunar_start + encodedParams) as response:
            if response.getcode() == 200:
                source = response.read()        
                data = json.loads(source)
                print(data['usage'])
            else:
                print('An error occurred while attempting to retrieve data from the API.')

    df_group_z = pd.DataFrame()
    df_group_price = pd.DataFrame()
    for coin in data['data']:

        coinName = coin['symbol'] + '-'+ coin['name']
        print(coinName)
        
        try:
            df = pd.DataFrame(coin['timeSeries'])[['time', 'close']]
        except:
            print(coinName, ' skippped, missing df cols')
            continue

        if df.isnull().sum().sum() > 0:
            print(coinName, ' is missing')
            continue

        # https://stackoverflow.com/questions/19231871/convert-unix-time-to-readable-date-in-pandas-dataframe
        df['time'] = pd.to_datetime(df['time'],unit='s')  

        df.columns = ['time', coinName]
        df = df.set_index('time')

        df_group_price = pd.concat([df_group_price, df], axis=1)

        # df = df.dropna(axis='columns')
        df[coinName] = stats.zscore(df[coinName])

        #display(df)
        df_group_z = pd.concat([df_group_z, df], axis=1)

    df_full_z = pd.concat([df_full_z, df_group_z], axis=1)
    df_full_price = pd.concat([df_full_price, df_group_price], axis=1)

df_full_z # 720 x 1118 cols - 50 coins
        # 720 x 1095 - 25 coins
        # 720 rows × 1112 50 coins, 15 seconds
        # 720 rows × 1117 50 coins, 60 seconds 

CWS-Crowns
DIGG-DIGG
FEG-Feed Every Gorilla
FL-Freeliquid
FL-Freeliquid  is missing
FWT-Freeway Token
FXF-Finxflo
GLCH-Glitch Protocol
HELMET-Helmet Insure
IBFKFT-Istanbul Basaksehir Fan Token
IBFKFT-Istanbul Basaksehir Fan Token  is missing
ID-Everest
INFI-Insured Finance
JGN-Juggernaut
JULD-JulSwap
KEBAB-Kebab Token
KSP-KlaySwap Protocol
MBX-MobieCoin
MSS-Monster Slayer Share
MSS-Monster Slayer Share  is missing
MWBTC-MetaWhale BTC
MWBTC-MetaWhale BTC  is missing
MWG-Metawhale Gold
MWG-Metawhale Gold  is missing
NOVFT-Novara Calcio Fan Token
NOVFT-Novara Calcio Fan Token  is missing
OMI-ECOMI
OPCT-Opacity
OPIUM-Opium
PAID-PAID Network
PAID-PAID Network  is missing
PAINT-Paint
PGU-Polyient Games Unity
PGU-Polyient Games Unity  is missing
POOLZ-Poolz.Finance
QUICK-Quickswap
SFP-SafePal.io
STM-Streamity
STM-Streamity  is missing
SURF-SURF.Finance
SURF-SURF.Finance  is missing
SX-SportX
THFT-Team Heretics Fan Token
THFT-Team Heretics Fan Token  is missing
TORN-Tornado Cash
TROP-Interop
T

,ADA-Cardano,ATOM-Cosmos,BAT-Basic Attention Token,BCD-Bitcoin Diamond,BCH-Bitcoin Cash,BNB-Binance Coin,BSV-Bitcoin SV,BTC-Bitcoin,BTG-Bitcoin Gold,BTT-BitTorrent,...,SIL-Sister In Law,SPE-SavePlanetEarth,SRX-StorX Token,STACK2-StackOS,UFO2-The Truth,VPP-Virtue Poker,WAIV-Waiv,WEYU-WEYU,WOOF-Shibance Token,ZENC-Zenccoin
time,,,,,,,,,,,,,,,,,,,,,
2019-09-25,-0.682868,-0.867196,-0.763299,-0.603451,-0.736364,-0.640305,-1.591539,-0.804713,-0.583833,-0.474735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-26,-0.684213,-0.877807,-0.790771,-0.677764,-0.797760,-0.645266,-1.644693,-0.826891,-0.589445,-0.489073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-27,-0.682724,-0.869998,-0.743200,-0.635144,-0.771423,-0.642585,-1.623089,-0.820606,-0.588217,-0.485728,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-28,-0.683146,-0.870142,-0.753507,-0.645852,-0.739701,-0.642299,-1.622081,-0.818474,-0.585650,-0.482860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-29,-0.685193,-0.881016,-0.770767,-0.688310,-0.781189,-0.645268,-1.656551,-0.827682,-0.599463,-0.491463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-09,2.784102,2.614350,1.229424,1.152974,1.205238,1.695164,-0.447151,1.355093,1.545102,1.275121,...,1.286475,-0.422121,1.389849,-1.761945,-0.829180,0.430398,-1.243069,-0.570842,-1.001474,0.737034
2021-09-10,2.600751,2.991723,1.082902,1.011615,1.064551,1.585792,-0.517032,1.272118,1.424068,1.165839,...,1.205009,-1.302170,1.027677,-0.568971,-1.165651,-0.350480,-1.344945,-1.131891,-1.472715,0.765938
2021-09-11,2.958273,2.687460,1.114224,1.033970,1.098040,1.604340,-0.472490,1.291878,1.421768,1.182917,...,1.087712,-1.362123,1.171771,-0.679433,-1.271517,-0.198654,-1.482033,-0.748974,-1.489082,0.702807


In [ ]:
df_full_price

,ADA-Cardano,ATOM-Cosmos,BAT-Basic Attention Token,BCD-Bitcoin Diamond,BCH-Bitcoin Cash,BNB-Binance Coin,BSV-Bitcoin SV,BTC-Bitcoin,BTG-Bitcoin Gold,BTT-BitTorrent,...,SIL-Sister In Law,SPE-SavePlanetEarth,SRX-StorX Token,STACK2-StackOS,UFO2-The Truth,VPP-Virtue Poker,WAIV-Waiv,WEYU-WEYU,WOOF-Shibance Token,ZENC-Zenccoin
time,,,,,,,,,,,,,,,,,,,,,
2019-09-25,0.039054,2.299140,0.161281,0.477745,228.211684,16.102139,86.676224,8464.917221,7.770585,0.000459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-26,0.038091,2.224171,0.152301,0.401846,214.492623,15.240224,83.459686,8075.227289,7.606122,0.000429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-27,0.039157,2.279343,0.167851,0.445376,220.377650,15.705934,84.767013,8185.655469,7.642106,0.000436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-28,0.038855,2.278325,0.164482,0.434439,227.466114,15.755632,84.828018,8223.120452,7.717349,0.000442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-29,0.037390,2.201502,0.158840,0.391076,218.195478,15.239825,82.742076,8061.324161,7.312555,0.000424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-09,2.520531,26.895262,0.812672,2.271634,662.070099,421.882592,155.927984,46415.265094,70.158289,0.004120,...,152.174823,3.649990e-08,3.819998,0.021284,0.000003,0.598414,0.000890,0.034044,0.012179,0.009995
2021-09-10,2.389298,29.561298,0.764776,2.127259,630.632990,402.879667,151.699189,44957.294816,66.611428,0.003892,...,150.794554,3.163438e-08,3.591870,0.025927,0.000002,0.539502,0.000787,0.025910,0.008653,0.010042
2021-09-11,2.645193,27.411769,0.775015,2.150092,638.116172,406.102287,154.394610,45304.506965,66.544032,0.003927,...,148.807197,3.130292e-08,3.682633,0.025497,0.000002,0.550957,0.000648,0.031462,0.008531,0.009939


In [ ]:
print(data['config']['data_points'], ' days for: ', data['config']['symbol'])
# supervised... price prediction or predictict 'successful' coin
# unsupervised, get a few tsfresh features from a few cols of a coin, then cluster
# need to find most impactful tsfresh
data['config']

720  days for:  'btc','eth','xrp','ltc','bch','bnb','usdt','eos','bsv','xlm','ada','trx','xmr','leo','dash','neo','miota','link','atom','xtz','etc','xem','mkr','ont','cro','zec','vsys','usdc','doge','btg','bat','vet','egt','qtum','dcr','ht','hedg','omg','tusd','pax','rvn','nano','lsk','btt','bcd','hot','kcs','waves','zrx','npxs','rep','icx','dgb','mona','kmd','hc','bts','aoa','iost','bcn'


{'data': 'assets',
 'data_points': 720,
 'interval': 'day',
 'symbol': "'btc','eth','xrp','ltc','bch','bnb','usdt','eos','bsv','xlm','ada','trx','xmr','leo','dash','neo','miota','link','atom','xtz','etc','xem','mkr','ont','cro','zec','vsys','usdc','doge','btg','bat','vet','egt','qtum','dcr','ht','hedg','omg','tusd','pax','rvn','nano','lsk','btt','bcd','hot','kcs','waves','zrx','npxs','rep','icx','dgb','mona','kmd','hc','bts','aoa','iost','bcn'",
 'time_series_indicators': 'close'}

In [ ]:
#data

In [ ]:
# df_full = pd.DataFrame()
# for coin in data['data']:

#     coinName = coin['symbol'] + '-'+ coin['name']
#     print(coinName)
    
#     try:
#         df = pd.DataFrame(coin['timeSeries'])[['time', 'close']]
#     except:
#         print(coinName, ' skippped, missing cols')
#         continue

#     if df.isnull().sum().sum() > 0:
#         print(coinName, ' is missing')
#         continue

#     # https://stackoverflow.com/questions/19231871/convert-unix-time-to-readable-date-in-pandas-dataframe
#     df['time'] = pd.to_datetime(df['time'],unit='s')  

#     df.columns = ['time', coinName]
#     df = df.set_index('time')

#     # df = df.dropna(axis='columns')
#     df[coinName] = stats.zscore(df[coinName])

#     #display(df)
#     df_full = pd.concat([df_full, df], axis=1)
    


# data['data']
# df_full

ADA-Cardano
AOA-Aurora
ATOM-Cosmos
BAT-Basic Attention Token
BCD-Bitcoin Diamond
BCH-Bitcoin Cash
BCN-Bytecoin
BNB-Binance Coin
BSV-Bitcoin SV
BTC-Bitcoin
BTG-Bitcoin Gold
BTS-BitShares
BTT-BitTorrent
CRO-Crypto.com Chain
DASH-Dash
DCR-Decred
DGB-DigiByte
DOGE-Dogecoin
EGT-Egretia
EOS-EOS
ETC-Ethereum Classic
ETH-Ethereum
HC-HyperCash
HEDG-HedgeTrade
HEDG-HedgeTrade  is missing
HOT-Holo
HT-Huobi Token
ICX-ICON
IOST-IOStoken
KCS-KuCoin Shares
KMD-Komodo
LEO-UNUS SED LEO
LEO-UNUS SED LEO  is missing
LINK-Chainlink
LSK-Lisk
LTC-Litecoin
MIOTA-IOTA
MKR-Maker
MONA-MonaCoin
NANO-Nano
NEO-NEO
NPXS-Pundi X
OMG-OMG Network
ONT-Ontology
PAX-Paxos Standard
QTUM-Qtum
REP-Augur
REP-Augur  is missing
RVN-Ravencoin
TRX-TRON
TUSD-TrueUSD
USDC-USD Coin
USDT-Tether
VET-VeChain Thor
VSYS-V Systems
WAVES-Waves
XEM-NEM
XLM-Stellar
XMR-Monero
XRP-XRP
XTZ-Tezos
ZEC-Zcash
ZRX-0x


,ADA-Cardano,AOA-Aurora,ATOM-Cosmos,BAT-Basic Attention Token,BCD-Bitcoin Diamond,BCH-Bitcoin Cash,BCN-Bytecoin,BNB-Binance Coin,BSV-Bitcoin SV,BTC-Bitcoin,...,VET-VeChain Thor,VSYS-V Systems,WAVES-Waves,XEM-NEM,XLM-Stellar,XMR-Monero,XRP-XRP,XTZ-Tezos,ZEC-Zcash,ZRX-0x
time,,,,,,,,,,,,,,,,,,,,,
2019-09-23,-0.669558,1.261523,-0.831469,-0.682179,-0.486674,-0.448806,0.302042,-0.617569,-1.163620,-0.730164,...,-0.679108,5.154857,-0.731528,-0.746096,-0.741090,-0.732742,-0.506211,-1.323919,-0.721982,-0.812278
2019-09-24,-0.681648,0.999170,-0.881080,-0.773700,-0.616131,-0.755271,-0.157426,-0.637323,-1.612530,-0.796393,...,-0.689614,3.224696,-0.753283,-0.784576,-0.803844,-0.832892,-0.598695,-1.446209,-0.862029,-0.849428
2019-09-25,-0.679936,0.905284,-0.870698,-0.758951,-0.599723,-0.732504,-0.019304,-0.636031,-1.585346,-0.800204,...,-0.688982,2.910532,-0.749691,-0.769879,-0.781306,-0.822272,-0.567463,-1.474314,-0.832040,-0.803535
2019-09-26,-0.681294,0.741203,-0.881511,-0.786447,-0.674108,-0.793960,-0.070043,-0.641006,-1.638374,-0.822414,...,-0.691102,2.968795,-0.753750,-0.758642,-0.774532,-0.848374,-0.576235,-1.478484,-0.878830,-0.792350
2019-09-27,-0.679790,0.888905,-0.873553,-0.738834,-0.631447,-0.767598,-0.050311,-0.638318,-1.616821,-0.816120,...,-0.689205,3.021214,-0.752081,-0.762567,-0.770629,-0.844560,-0.573920,-1.447632,-0.838585,-0.800423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-07,2.829985,-0.663951,1.854035,1.140852,1.234775,1.278478,0.987176,1.710010,-0.418390,1.400351,...,1.552244,-0.680091,2.606611,0.318555,0.934049,1.291235,1.900646,1.233619,0.892005,0.810368
2021-09-08,2.767534,-0.674503,1.791437,1.114448,1.148109,1.226026,0.842036,1.685937,-0.433366,1.352141,...,1.463042,-0.677799,2.636584,0.292860,0.897920,1.227980,1.816996,1.171190,0.872145,0.792319
2021-09-09,2.820002,-0.673780,2.676860,1.235575,1.158372,1.211027,0.573270,1.706332,-0.443579,1.362879,...,1.514200,-0.670878,3.097190,0.320534,0.902939,1.268545,1.770574,2.084434,0.907253,0.909931


In [ ]:
df_full_z.to_csv('lunar_unsup_prices_z_50.csv', index=True)
df_full_price.to_csv('lunar_unsup_prices_50.csv', index=True)

In [ ]:
df_full_z.info(verbose=True, null_counts=True)

 117   POWR-Power Ledger                                    720 non-null    float64
 118   PPT-Populous                                         720 non-null    float64
 119   QASH-QASH                                            720 non-null    float64
 120   QKC-QuarkChain                                       720 non-null    float64
 121   RDD-ReddCoin                                         720 non-null    float64
 122   SNX-Synthetix Network Token                          720 non-null    float64
 123   TFUEL-Theta Fuel                                     720 non-null    float64
 124   TOMO-TomoChain                                       720 non-null    float64
 125   TRUE-TrueChain                                       720 non-null    float64
 126   VERI-Veritaseum                                      720 non-null    float64
 127   WAN-Wanchain                                         720 non-null    float64
 128   WICC-WaykiChain                                      720 non-null    

In [ ]:
df_full_no_na = df_full_z.dropna(axis='columns')
df_full_no_na.shape

(720, 502)

In [ ]:
df_full_price.dropna(axis='columns').shape

(720, 502)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=465bb6a2-b771-4fe2-b2d2-aed723f7eb7d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>